In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader

urls = [
    "https://www.hindustantimes.com/world-news/us-news/donald-trump-to-announce-new-reciprocal-tariffs-shortly-ahead-of-meeting-with-pm-modi-101739471114365.html",
    "https://www.bloomberg.com/news/newsletters/2025-02-14/president-trump-s-tariff-plan-is-pushed-to-april",
    "https://m.economictimes.com/news/economy/foreign-trade/trump-brandishes-his-big-weapon-what-are-reciprocal-tariffs/articleshow/118220603.cms",
    "https://www.moneycontrol.com/news/opinion/what-is-donald-trump-s-reciprocal-tariff-plan-12941110.html"
]

loader =  UnstructuredURLLoader(urls = urls )
data = loader.load()

KeyboardInterrupt: 

In [2]:
len(data)

4

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))



Total number of documents:  34


In [4]:
docs[3]

Document(metadata={'source': 'https://m.economictimes.com/news/economy/foreign-trade/trump-brandishes-his-big-weapon-what-are-reciprocal-tariffs/articleshow/118220603.cms'}, page_content='ET OnlineLast Updated: Feb 14, 2025, 12:20:00 AM IST\n\nRate Story\n\nFollow us\n\nShare\n\nFont Size\n\nAbcSmall\n\nAbcMedium\n\nAbcLarge\n\nSave\n\nPrint\n\nComment\n\nSynopsis\n\nJust before meeting Narendra Modi, US President Donald Trump announced plans to impose reciprocal tariffs on countries matching their tariffs on US goods. This move, intended to address trade imbalances, could stir international tensions and impact global economic growth. Reciprocal tariffs are expected to align American tariffs with those imposed by other countries.')

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

vector = embeddings.embed_query("hello, world!")
vector[:5]

C:\Users\tjwin\AppData\Local\Temp\ipykernel_228\2980807805.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\tjwin\AppData\Local\Temp\ipykernel_228\2980807805.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.i

[0.034922655671834946,
 0.018830018118023872,
 -0.017854738980531693,
 0.0001388440141454339,
 0.0740736871957779]

In [6]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=docs, embedding=HuggingFaceEmbeddings())

C:\Users\tjwin\AppData\Local\Temp\ipykernel_228\1700344016.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = Chroma.from_documents(documents=docs, embedding=HuggingFaceEmbeddings())


In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Trump and tariff")

In [8]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [9]:
model_id = "tiiuae/falcon-7b"

text_generation_pipeline = pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, max_new_tokens=400, device=0)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tjwin\.cache\huggingface\hub\models--tiiuae--falcon-7b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
  warnings.warn(
Error while downloading from https://cdn-lfs.hf.co/repos/80/bd/80bd1e4a99ab55602d887f91215879d7

ValueError: Could not load model tiiuae/falcon-7b with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.falcon.modeling_falcon.FalconForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\urllib3\response.py", line 754, in _error_catcher
    yield
  File "c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\urllib3\response.py", line 879, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
  File "c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\urllib3\response.py", line 862, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "c:\Users\tjwin\anaconda3\envs\gem_learn\lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
  File "c:\Users\tjwin\anaconda3\envs\gem_learn\l

In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain


In [ ]:
rag_chain.invoke("what is trumps tariff plan")

In [ ]:
question = "2024 Budget Highlights"
response = rag_chain.invoke(question)

# Making the response readable
response = response.replace("</s>", "").strip()
print("Response:", response)